<a href="https://colab.research.google.com/github/abhishekkumawat23/build-llm-from-scratch/blob/main/Build_a_large_language_model_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Large Language Model from Scratch

This notebook follows the [Build a Large Language Model (From Scratch)](https://sebastianraschka.com/llms-from-scratch/) book to build a large language model from scratch.

Wherever needed, the notebook explains the relevant theory and underlying concepts, focusing on learning through hands-on implementation.

---

## What is an LLM (Large Language Model)?

**Large Language Models (LLMs)** can be of various types, but the most common ones are **Generative LLMs**. A Generative LLM is a deep neural network model that, when given a piece of text (whether a word, line, or paragraph), generates the next word (or token, to be precise) that should follow that text.

### Example

- Given `The cat sat`, the LLM might generate `on`
- Given `The cat sat on`, the LLM might generate `the`
- Given `The cat sat on the`, the LLM might generate `mat`

Since the model can generate the next word after any given text, if we pass `The cat sat` and call the model multiple times—each time appending the word it generated in the previous iteration—we can generate a significant chunk of text. For example, starting with just `The cat sat`, we can generate `The cat sat on the mat and started playing.`

### How It Works

- `The cat sat` → `The cat sat on`
- `The cat sat on` → `The cat sat on the`
- `The cat sat on the` → `The cat sat on the mat`
- `The cat sat on the mat` → `The cat sat on the mat and`
- `The cat sat on the mat and` → `The cat sat on the mat and started`
- `The cat sat on the mat and started` → `The cat sat on the mat and started playing`

End users don't need to call the model repeatedly after each word—the LLM wraps this logic internally, calling the underlying model multiple times and appending each generated word to the next iteration. This is why such LLMs are also called **Auto-Regressive** models.

---

### TODO
Introduce additional foundational concepts such as:
- Deep neural networks
- Transformer layers
- Attention mechanisms
- Encoder-only, decoder-only, and encoder-decoder architectures
- Tokenizers
- Embeddings
- Difference between inference and training
- During training, the entire input is processed at once, while during inference, text is generated one token at a time
- KV caching

*Note: Some of these concepts may be introduced at later stages in the notebook.*

---

# Text Processing

## What is an Embedding?

LLMs don't understand text the way humans do—they understand numbers. Therefore, we need to:
- Convert the input text into a numerical representation before passing it to the LLM
- When the LLM predicts the next word, convert its numerical output back into text

This numerical representation of text is called an **embedding**. An embedding is not a single number, but rather a vector (a list) of floating-point numbers. The length of this vector depends on the LLM architecture and can be 32, 64, 768, 4096, or more. The length of the embedding vector is called the **embedding dimension**.

## What to Embed: Entire Input Text or Individual Words?

If we want to pass `The cat sat` to the LLM, should we convert the entire text to a single embedding vector, or should we convert each word to its own embedding vector?

This depends on the type of LLM being used, but most generative LLMs use individual embeddings for each word-like unit rather than sentence-level embeddings. So `The cat sat` would be converted to 3 embedding vectors, one for each word. As input, a generative LLM receives a sequence of embedding vectors, and as output it returns another sequence of embedding vectors representing the generated text `on the mat and started playing` (where each word was generated auto-regressively, one by one, internally).

However, the reality is more nuanced: LLMs don't actually use *word-level* embeddings, but rather *token-level* subword embeddings. Here, a *token* can represent a complete word, a subword, a space, a special character, or a regular character

## Why Tokens Instead of Words?

Words are typically composed of root words combined with prefixes and suffixes (affixes). If we created an embedding vector for each complete word, we would need an enormous number of embedding vectors. For example, `help`, `helper`, `helpless`, `helpful`, and `unhelpful` are 5 different words all derived from the root word `help`.

Instead, if we split these words into reusable subword tokens like `help`, `er`, `less`, `ful`, and `un`, we only need 5 embedding vectors—but now these subwords can be reused to represent many other words (like `teach`+`er`, `hope`+`less`, `use`+`ful`, `un`+`done`). This significantly reduces the total number of embedding vectors needed for the training dataset. These reusable subword units—such as `help`, `er`, `less`, `ful`, and `un`—are called **tokens**.

## Token Vocabulary

If we collect all the unique tokens from the training dataset, we have a token **vocabulary**. Each token in the vocabulary is assigned a unique identifier called a **token ID**.

## Tokenizer: Converting Between Text and Token IDs

A **tokenizer** is responsible for converting between text and token IDs:
- **Encoding**: The tokenizer converts text into token IDs. These token IDs are then converted to embedding vectors by the embedding layer. The LLM processes these input embedding vectors and generates output embedding vectors, which are converted back to token IDs.
- **Decoding**: The tokenizer converts the output token IDs back into human-readable text.

A tokenizer is capable of performing both encoding and decoding operations.

## Embedding vs Token IDs

One might think that as we can get token IDs from tokens, we already got a numerical representation of tokens. Then what's the need of converting them to embedding vectors? Why not just pass the token ids (numerical representation) to LLMs?

Emebdding vectors are more than just numerical reprsentations. They somehow also contains relationships and pattern between words. Similar embedding vectors are near to each other, and dissimilar are far. For example, embedding vectors of *cat*, *dog* will be near to each other than *car*. By *near*, I mean angle between *cat* and *dog* embedding vectors is less than angle between between *cat* and *car* in the embedding vector space. In other words, emebdding vectors holds semantic relationship between words. We will cover this later on that how emebdding vectors holds these semantic relationships and complex patterns. For now, we just wanted to state that we still need embedding vectors even though we have token ids as numerical representation of tokens.

---

## Loading the Training Dataset

Before diving deeper into converting text to tokens and then tokens to embedding vectors for input to an LLM, let's first load our dataset.

To build our LLM, we will use a very small dataset: a short story by **Edith Wharton** titled **The Verdict**.

Download the text from https://en.wikisource.org/wiki/The_Verdict and save it as a file named **the-verdict.txt**.

In [66]:
# Load the file and read content

with open('the-verdict.txt', 'r', encoding='utf-8') as file:
  raw_text = file.read()

print(f'Total number of characters: {len(raw_text)}')
print(raw_text[:99])


Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


## Simple Token Vocabulary - Word-Based

- Let's implement a simple token vocabulary. We will treat each word as a token, without any fancy word splitting.
- Since special characters like spaces, commas, periods, and exclamation marks are typically attached to words, let's split by these characters to extract them as separate tokens.

In [67]:
# Simple token vocab - v1 - word based

import re

tokens = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
vocab = sorted(set([token for token in tokens if token.strip()]))

## Simple Tokenizer - v1 - Word-Based

- Let's implement a simple text tokenizer called `SimpleTokenizerV1` using the word-based token vocabulary we created earlier.
- In the `encode` method, we will convert the text into a list of token IDs. For splitting the text, we will use the same regex pattern used to create the vocabulary, ensuring consistency.
- In the `decode` method, **TODO**

**Limitations:**
- Since the vocabulary contains only words from the training dataset, this tokenizer will **throw an error when it encounters an unknown word** during inference.
- Given text like `It's the last he painted.`, the encoding and subsequent decoding will return ` It' s the last he painted.` with an extra space at the start of the decoded text and an extra space before the `s` in `It's`. This occurs due to how the regex splits tokens and how they are rejoined during decoding.

In [68]:
# Simple tokenizer v1 - word are tokens here
class SimpleTokenizerV1:
  def __init__(self, vocab):
    self.token_to_id = {token: id for id, token in enumerate(vocab)}
    self.id_to_token = {id: token for token, id in self.token_to_id.items()}

  def encode(self, text):
    tokens = re.split(r'([,.?_!"()\']|--|\s)', text)
    token_ids = [self.token_to_id[token] for token in tokens if token.strip()]
    return token_ids

  def decode(self, token_ids):
    tokens = [self.id_to_token[token_id] for token_id in token_ids]
    text = ' '.join(tokens)
    # Joining by space caused space before each special character as well. Remove it.
    text = re.sub(r'\s+([,.?_!"()\']|--)', r'\1', text)
    return text



In [69]:
# Testing the simple tokenizer v1.
tokenizer = SimpleTokenizerV1(vocab)
text_to_encode = 'It\'s the last he painted.'
token_ids = tokenizer.encode(text_to_encode)
print(f'Encoded token ids for {text_to_encode}: {token_ids}')

decoded_text = tokenizer.decode(token_ids)
print(f'Decoded text for {token_ids}: {decoded_text}')

Encoded token ids for It's the last he painted.: [58, 2, 872, 1013, 615, 541, 763, 7]
Decoded text for [58, 2, 872, 1013, 615, 541, 763, 7]: It' s the last he painted.


## Simple Tokenizer - v2 - Word Based

**Adding the `<|unk|>` token:** The previous version threw an error when it encountered unknown words. We can introduce a special token to represent any unknown word. Let's call this token `<|unk|>`.

**Adding the `<|endoftext|>` token:** As we know, we pass a list of token IDs to the LLM. The maximum length of this list is called `max_seq_len`. We will later learn that computationally, it costs roughly the same whether we pass 1 token ID or `max_seq_len` token IDs to the LLM. Since the computational cost is similar, it makes sense during training to always pass inputs of length `max_seq_len` to maximize efficiency.

To achieve this, we can concatenate multiple lines or sentences until we reach `max_seq_len`, then pass that combined input to the LLM. However, we need the LLM to treat these concatenated lines as separate and unrelated, so that text generation isn't negatively affected by the arbitrary concatenation. For this purpose, we can add a special token called `<|endoftext|>` between concatenated lines. We can then train the LLM to understand that segments separated by `<|endoftext|>` are independent and unrelated to each other.

In [70]:
# Add `<|unk|>` and `<|endoftext|>` tokens in the vocab
vocab.extend(['<|unk|>', '<|endoftext|>'])
print(vocab[-5:])

# Create tokenize with these special tokens support
class SimpleTokenizerV2:
  def __init__(self, vocab):
    self.token_to_id = {token:id for id, token in enumerate(vocab)}
    self.id_to_token = {id:token for id, token in enumerate(vocab)}

  def encode(self, text):
    tokens = re.split(r'([,.?_!"()\']|--|\s)', text)
    tokens = [token for token in tokens if token.strip()]
    tokens = [token if token in self.token_to_id else '<|unk|>' for token in tokens]
    token_ids = [self.token_to_id[token] for token in tokens]
    return token_ids

  def decode(self, token_ids):
    tokens = [self.id_to_token[token_id] for token_id in token_ids]
    text = ' '.join(tokens)
    text = re.sub(r'\s+([,.?_!"()\']|--)', r'\1', text)
    return text

['younger', 'your', 'yourself', '<|unk|>', '<|endoftext|>']


In [71]:
# Lets test with tokenizer

tokenizer = SimpleTokenizerV2(vocab)
token_ids = tokenizer.encode('Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.')
print(tokenizer.decode(token_ids))


<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


---

## Simple Byte based Tokenizer

**Limitations of simple tokenizer:**

In the simple tokenizers we implemented, there are many limitations:
- We are considering words as tokens but as we discussed earlier example of `help`, `helper`, `helpless`, `helpful`, and `unhelpful` where we if instead of these, we stored `help`, `er`, `less`, `ful` and `un` as tokens, we would end up storing way few tokens as these common prefix and suffix we separated can be used to create many other words.
- Even though we are handling unknown word with special token `<|unk|>` and not erroring out, we are not passing the actual unkown word to the LLM, and thus even if LLM know how to make predictions with the unknown words(Example: typos), it can't do anything as its not getting that unknown word. Thus, Tokenizer should be able to create tokens for unknown words.

**Tokenizer made out of byte tokens:**

As we know that at the very core, each word, character, symbol, special tokens are made out of bytes. And how many bytes are possible? 256 (2^8 different values). So if we create a token vocabulary with all these 256 bytes in it, we can convert any word, character, symbol to tokens. This way we don't need to think about any unknown words.

**Do we need to implement byte tokenizer?:**

Python text already has encode and decode methods which encodes text to bytes and then decodes to text, so we don't need to implement a byte tokenizer. But lets still implement one because we are going to make it more complicated by adding many optimization in it.

**Limitations:**

In below tokenizer, number of tokens are only 256 so very small number of token. In addition, there is no problem of unknown words as all words can be created from the 256 bytes. There what are the limitations?
- Number of tokens required to represent a single word will be huge. Even a simple english word like `implement` now needs 9 tokens just to reprsent 1 word. LLMs have a limit on number of tokens it can take called `max_seq_length`. As we have increased number of tokens needed per word, LLM can take way smaller text/words in this case.
- As we have only 256 tokens, corresponding emebdding vectors will only be 256. As we know that embedding vectors somehow holds semnatic relationships as well, as we now have only 256 such vectors, the amount of relationships it can hold will be limited.

In [72]:
class SimpleByteTokenizer:
  def __init__(self):
    self.token_to_id = {bytes([id]): id for id in range(256)}
    self.id_to_token = {id: token for token, id in self.token_to_id.items()}

  def encode(self, text):
    tokens = text.encode('utf-8') # Using this method, we get the bytes
    token_ids = [self.token_to_id[bytes([token])] for token in tokens]
    return token_ids

  def decode(self, token_ids):
    tokens = [self.id_to_token[token_id] for token_id in token_ids]
    all_bytes = b''.join(tokens)
    return all_bytes.decode('utf-8', errors='replace')

In [73]:
tokenizer = SimpleByteTokenizer()
token_ids = tokenizer.encode('Hello how are you?')
print(token_ids)
print(tokenizer.decode(token_ids))


[72, 101, 108, 108, 111, 32, 104, 111, 119, 32, 97, 114, 101, 32, 121, 111, 117, 63]
Hello how are you?


## Simple Byte Pair Encoding Tokenizer

We can build on top of our existing simple byte tokenizer to address its limitations. We are going to build a tokenizer called **Byte Pair Encoding** Tokenizer aka **BPE** Tokenizer. GPT like LLMs use this BPE tokenizer. Here for simplicity we will implement a simple version.

**Creating the vocab:**
- Go through entire training dataset and split it into bytes to get `tokens`.
- Create a frequency map of consecutive byte pairs. Find out the byte pair with max frequency.
- Merge this pair and add the merged bytes in the `token_to_id` and `id_to_token` map.
- Also go through the `tokens` list and replace the byte pair with single merged bytes.
- Now repeat this process until you reach a vocab of size you want. For example if you want vocab of 5000 size, then repeat the process for 5000-256 times as we already had 256 values in the vocab.
- Repeating this process allows vocab to be added with most common sub-words, word (in merged byte format).

**Encoding:**
- Apply greedy longest match approach to split text into minimal tokens within O(n) time

**Decoding:**
- Decoding is pretty simple as same as before.

In [79]:
from collections import Counter

class SimpleBPETokenizer:
  def __init__(self, vocab_size, freq_threshold):
    self.token_to_id = {bytes([id]): id for id in range(256)}
    self.id_to_token = {id: token for token, id in self.token_to_id.items()}
    self.vocab_size = vocab_size
    self.freq_threshold = freq_threshold

  def train(self, text):
    # Pass entire training text as text so that we can create tokens for common
    # sub-words, words etc.
    tokens = text.encode('utf-8')
    tokens = [bytes([token]) for token in tokens]

    num_merges = self.vocab_size - len(self.token_to_id)
    for _ in range(num_merges):
      # Find best token pair which comes together most frequently.
      pairs = Counter()
      for idx in range(len(tokens)-1):
        pair = (tokens[idx], tokens[idx+1])
        pairs[pair] += 1
      if not pairs:
        # Note: Train text had very few tokens that we have exhausted them
        # before creating the vocab of needed size.
        break
      best_pair = max(pairs, key=pairs.get)
      best_pair_merged = best_pair[0] + best_pair[1]

      # If best_pair's frequency doesn't cross the frequency threshold, then stop.
      if pairs[best_pair] < self.freq_threshold:
        break

      # Add best pair in our token_to_id and id_to_token maps.
      best_pair_idx = len(self.token_to_id) # As we are adding as last, index will be last.
      self.token_to_id[best_pair_merged] = best_pair_idx
      self.id_to_token[best_pair_idx] = best_pair_merged

      # Merge this best pair in tokens so that next iteration can repeat the
      # process but where tokens have this best pair merged. This way, iteration
      # by iteration, we would have merged common sub-words, words, etc.
      new_tokens = []
      idx = 0
      while idx < len(tokens):
        if idx < len(tokens) - 1 and (tokens[idx], tokens[idx+1]) == best_pair:
          new_tokens.append(best_pair_merged)
          idx += 2
        else:
          new_tokens.append(tokens[idx])
          idx += 1
      tokens = new_tokens

  def vocab(self):
    return list(self.token_to_id.keys())

  def encode(self, text):
    # Apply greedy longest match approach to split text into minimal tokens within O(n) time
    # TODO: is there stringbuilder kinda thing in python?
    text_bytes = [bytes([token]) for token in text.encode('utf-8')]

    start = 0
    end = 0
    current_token = b''
    tokens = []
    while end <= len(text_bytes):
      if end == len(text_bytes):
        tokens.append(current_token)
        break
      new_token = current_token + text_bytes[end]
      if new_token in self.token_to_id:
        current_token = new_token
      else:
        tokens.append(current_token)
        # print(f'Tokens: {tokens}')
        start = end
        current_token = text_bytes[end]
      end += 1

    token_ids = [self.token_to_id[token] for token in tokens]
    return token_ids

  def decode(self, token_ids):
    tokens = [self.id_to_token[token_id] for token_id in token_ids]
    all_bytes = b''.join(tokens)
    return all_bytes.decode('utf-8', errors='replace')

In [80]:
# Train tokenizer
vocab_size = 500
freq_threshold = 5
tokenizer = SimpleBPETokenizer(vocab_size, freq_threshold)
# print(raw_text[:99])
tokenizer.train(raw_text)

# Print vocab
vocab = tokenizer.vocab()
print(vocab[-10:])
print(f'Vocab size: {len(vocab)}')

[b'for ', b'ould', b'tch', b'gre', b'pr', b'ould have ', b'--t', b'cou', b'ure ', b'k ']
Vocab size: 500


In [81]:
# Try encoding and decoding
token_ids = tokenizer.encode('I felt able to face the fact')
print(token_ids)
print(tokenizer.decode(token_ids))

[278, 102, 288, 259, 336, 353, 116, 273, 102, 300, 305, 262, 102, 300, 116]
I felt able to face the fact


## GPT's BPE Tokenizer

The tokenizer we wrote just gives a simple idea about Byte Pair Encoding works. It lacks lot of optimizations like:
- We don't want weird tokens spreading over spaces. So ideally we should have pre-tokenize using word split to create text chunks, and then built the byte pair encoding on each chunk.
- Our encode is not efficient. Even though we are using greedy approach, we are appending to string multiple times where string is immutable. This causes time complexity to increase.

These are just few missing points we discussed but there are many. So, its better to use **GPT's BPE tokenizer**. Alternatively, we can also use some other tokenizers like **SentencePiece** tokenizer.

In [83]:
import tiktoken

tokenizer = tiktoken.get_encoding('gpt2')
token_ids = tokenizer.encode('I felt able to face the fact')
print(token_ids)
print(tokenizer.decode(token_ids))

[40, 2936, 1498, 284, 1986, 262, 1109]
I felt able to face the fact


---

## Data Loader

To prepare data for LLM training, we need to a lot of things:
- Data sampling - i.e. Creating chunks of entire raw text so that each chunk can be passed as one input sequence to the LLM. Here we can use stride and sliding window concept to pick the samples.
- Creating input-target pairs
- Creating batches of inputs so that one entire batch can be passed to the LLM at a time
- Whether to shuffle data or not betweeb say epochs
- Whether to drop the last batch or not as it might contain less data and thus we can avoid unnecessary spikes.

We can write all these logics by ourselves and its not that difficult but we have to be cautius while sampling the data.

Instead, we can use torch's dataset and dataloader classes to define the dataset.

In [85]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]

  def __len__(self):
    return len(self.input_ids)